In [12]:
import os

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper 


### Arxiv Tool

In [6]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results= 1,doc_content_chars_max=200) 

arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
print(arxiv_tool.name)
arxiv_tool


arxiv


ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

### Wikipedia Tool

In [10]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results= 1,doc_content_chars_max=200) 

wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
print(wiki_tool.name)
wiki_tool

wikipedia


WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\krish naik GenAI Course\\chatbot_course\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [11]:
## Combining both the tools
tool = [arxiv_tool, wiki_tool]
tool

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\krish naik GenAI Course\\chatbot_course\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))]

### Lets try to create our agent

In [13]:
# Importing Groq model
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

llm = ChatGroq(model='Llama3-8b-8192', api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A55C133D30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A54BE23070>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
### Prompt template
from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

# We have selected a custom prompt from huggingface hub which will save the chat history as well 

c:\Users\sredekar\OneDrive - Interpublic\Desktop\Datascience\krish naik GenAI Course\chatbot_course\lib\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
## Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm,prompt=prompt, tools=tool)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.m

In [21]:
## to invoke anything from our agent we will have to create a Egent executor

from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=agent, tools=tool, verbose= True)
executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChu

In [24]:
## Invoking the tool 
executor.invoke({"input":"Tell me about machine learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton Univer
Invoking: `arxiv` with `{'query': 'Machine learning optimization'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton UniverMachine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make predictions, and improve their performance over 

{'input': 'Tell me about machine learning',
 'output': "Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make predictions, and improve their performance over time. Optimization is a critical component of machine learning, as it allows models to find the best possible solution to a problem given a set of constraints.\n\nThere are many different approaches to optimization in machine learning, including convex optimization, non-convex optimization, and online learning. Each of these approaches has its own strengths and weaknesses, and the choice of which one to use depends on the specific problem being addressed.\n\nSome common techniques used in optimization for machine learning include gradient descent, stochastic gradient descent, and Newton's method. These techniques can be used to minimize the loss function of a model, which is a measure of how well the model is performing.\n\nO

In [25]:
executor.invoke({"input":"Tell me about deep learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Deep learning'}`


Page: Deep learning
Summary: Deep learning is a subset of machine learning methods that utilize neural networks for representation learning. The field takes inspiration from biological neuroscience an
Invoking: `arxiv` with `{'query': 'Deep learning'}`


Published: 2018-05-22
Title: Opening the black box of deep learning
Authors: Dian Lei, Xiaoxiao Chen, Jianfei Zhao
Summary: The great success of deep learning shows that its technology contains
profou
Invoking: `arxiv` with `{'query': 'Deep learning tutorial'}`


Published: 2024-09-17
Title: Deep Learning with CNNs: A Compact Holistic Tutorial with Focus on Supervised Regression (Preprint)
Authors: Yansel Gonzalez Tejeda, Helmut A. Mayer
Summary: In this tutor
Invoking: `wikipedia` with `{'query': 'Convolutional neural network'}`


Page: Convolutional neural network
Summary: A convolutional neural network (CNN) is a regularized type of feed-forward neural network that learns fe

{'input': 'Tell me about deep learning',
 'output': 'Deep learning is a subset of machine learning methods that utilize neural networks for representation learning. The field takes inspiration from biological neuroscience and has achieved state-of-the-art results in various areas, such as computer vision, speech recognition, and natural language processing. Deep learning models are particularly useful when the input data is high-dimensional and complex, and when there are many features that are relevant to the problem at hand.\n\nA key component of deep learning is the convolutional neural network (CNN), which is a type of feed-forward neural network that learns features by itself via filter (or kernel) optimization. CNNs are particularly well-suited to image and video processing tasks, as they can automatically detect and extract features from images and videos, such as edges, shapes, and textures.\n\nIn addition to CNNs, other types of deep learning models include recurrent neural ne